In [1]:
import json, os
import pandas as pd
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.sequential import SequentialChain
from langchain.callbacks.manager import get_openai_callback
from PyPDF2 import PdfReader
from docx import Document

load_dotenv()  # take environment variables from .env

True

In [2]:
read_vocab = PdfReader("pdf_files/science_vocab_handout_4-6.pdf")

vocab_handout = ""
for i, page in enumerate(read_vocab.pages):
    content = page.extract_text()
    if content:
        vocab_handout += content

In [3]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    temperature=0.7,
    )

In [4]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "A.": "choice here",
            "B.": "choice here",
            "C.": "choice here",
            "D.": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "A.": "choice here",
            "B.": "choice here",
            "C.": "choice here",
            "D.": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "A.": "choice here",
            "B.": "choice here",
            "C.": "choice here",
            "D.": "choice here",
        },
        "correct": "correct answer",
    },
}

In [5]:
first_template = """
This is how you are to respond to this task: You are an expert seasoned examiner with the Apprenticeship and Industry Training (AIT) organization in Alberta, Canada. You have 20 years of experience 
preparing the preapprenticeship entrance exams for skilled trade participants. You are highly skilled and have high level expertise with minimal typographic mistakes or errors in the answers. You are 
skilled in generating multiple choice questions that contains both calculations, theory and conceptual questions. This tone must strongly reflect in the generated multiple choice questions. You are
provided with a vocabulary handout for the science subject. Using the {vocab_handout}, creatively generate mutiple choice questions having {number} questions. The quiz questions should conform to the 
standard of the AIT exam questions. The questions should be based on the {vocab_handout}. Each question must be unique and completely different from others. Format your response using the {response_json} 
template as a guide. Ensure to include some questions that will involve calculations using formulas and also conversion between metric/imperial units. Some questions should not be straightforward and 
should require the trade participants to think critically. Ensure that you cross-check the {vocab_handout} and provide the correct information for each question and the exact answer for each question. 
Use the {response_json} template to format your response. The total number of quesions must be {number}. Some examples of calculation questions and their corresponding answers are provided in the below:

**Specific Requirements:**

1. **Theory and Conceptual Questions**: Include questions that test theoretical knowledge and conceptual understanding.
2. **Critical Thinking**: Some questions should not be straightforward and should require the trade participants to think critically.
3. **Accuracy**: Ensure that you cross-check the {vocab_handout} and provide the correct information for each question and the exact answer for each question.
4. **Diversity**: Questions should cover a wide range of topics within the provided vocabulary handout to ensure a comprehensive assessment.

**Example calculation questions:**
- 1. A car has a driving force of 1200 N and a mass of 700 kg. Calculate the acceleration of the car.
    a. 1.714 m/s^2
    b. 0.583 m/s^2
    c. 840000 m/s^2
    d. 0.6 m/s^2
- 2. Find the pressure exerted by a force of 600 Newtons on an area of 30 square meters.
    a. 20 Newton per square meter
    b. 1800 Newton per square meter
    c. 0.05 Newton per square meter
    d. 180 Newton per square meter
- 3. A block of aluminum occupies a volume of 15.0 mL and has a mass of 40.5 kg. Calculate the density of aluminum?
    a. 607.5 grams per cubic centimeter
    b. 0.370 grams per cubic centimeter
    c. 2.7 grams per cubic centimeter
    d. 2700 grams per cubic centimeter
- 4. Which of the following does not affect the amount of heat a body will store?
    a. Density
    b. Altitude
    c. Mass
    d. Both (a) and (c)
- 5. Calculate the force required to accelerate a 1200 kg car from 5 m/s to 20 m/s in 5 seconds.
    a. 1200 N
    b. 2400 N
    c. 3600 N
    d. 4800 N
- Answers: 
    1. a
    2. a
    3. d
    4. b
    5. c

RESPONSE_JSON:
{{
    "1": {{
        "mcq": "multiple choice question",
        "options": {{
            "A.": "choice here",
            "B.": "choice here",
            "C.": "choice here",
            "D.": "choice here"
        }},
        "correct": "correct answer"
    }},
    "2": {{
        "mcq": "multiple choice question",
        "options": {{
            "A.": "choice here",
            "B.": "choice here",
            "C.": "choice here",
            "D.": "choice here"
        }},
        "correct": "correct answer"
    }},
    "3": {{
        "mcq": "multiple choice question",
        "options": {{
            "A.": "choice here",
            "B.": "choice here",
            "C.": "choice here",
            "D.": "choice here"
        }},
        "correct": "correct answer"
    }}
    // Continue the same structure for remaining questions
}}

Ensure that all your generated questions follow the {response_json} structure exactly.
"""

In [6]:
# first_template = """
# VOCAB:
# {vocab_handout}
# You are an examiner with the Apprenticeship and Industry Training (AIT) organization in Alberta, Canada. The {vocab_handout} is a vocabulary handout prepared for the 
# science subject. Using the {vocab_handout}, please generate multiple choice questions having {number} questions in total. The quiz questions should conform to the standard of the AIT exam 
# questions and **include at least 10 questions that require calculations** such as conversions between units, and applying formulas mentioned in the handout. 
# The questions should be based on the {vocab_handout}. Make sure the questions are not repeated and do format your response using the {response_json} template  
# as a guide. Ensure that you cross-check the {vocab_handout} and provide the correct information for each question and the exact correct answer for each question. Use the {response_json}
# template below to format your response. The total number of quesions must be {number}.

# **Here are some examples of calculation-based questions for your reference:**

# * If a car travels 100 kilometers in 2 hours, what is its speed in kilometers per hour?
# * A solution has a concentration of 5% salt. How many grams of salt are there in 100 grams of the solution?

# **Please make sure to include similar questions in your generated quiz.**
# """

In [7]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["vocab_handout", "number", "response_json"],
    template=first_template,
    )

In [8]:
quiz_chain = LLMChain(
    llm=llm,
    prompt=quiz_generation_prompt,
    output_key="quiz",
    verbose=True,
    )

In [9]:
second_template = """
This is the tone in which you are to respond to the assigned task: You are an expert exam reviewer with 20 years of experience working with the Apprenticeship and Industry Training (AIT) 
organization in Alberta, Canada. Your job is to ensure that the exam questions meet the required standard for pre-apprenticeship entrance exams. You are assigned to check for the correctness 
of each question, cross-check the answer, and carefully fact-check the information contained in the generated questions.

**Task Overview:**

Given some multiple-choice questions recently generated, you must:
1. Evaluate the complexity of each question and provide a complete analysis of the {quiz}. Use a maximum of 100 words for the complexity analysis.
2. Ensure that the questions include a balanced mix of calculation questions and theoretical/conceptual questions.
4. Check for the accuracy of the questions, correctness of formulas, and units used.
5. Verify the correctness of each provided answer.

**If the quiz does not meet the required standard:**

- Identify the questions that need to be changed.
- Generate new questions to replace the incorrect ones, ensuring they follow the existing structure and meet the specified requirements.
- Ensure the new questions are unique and diverse, covering a range of topics within the provided vocabulary handout.

**Evaluation Criteria:**

- Accuracy: Ensure all questions and answers are factually correct.
- Complexity: Analyze and ensure the questions are appropriately challenging.
- Calculation: Verify the inclusion and correctness of calculation-based questions.

**Quiz_MCQs:**
{quiz}

**Check from an Expert Reviewer:**
"""

In [10]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["quiz"],
    template=second_template,
    )

In [11]:
review_chain = LLMChain(
    llm=llm,
    prompt=quiz_evaluation_prompt,
    output_key="review",
    verbose=True,
    )

In [12]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["vocab_handout", "number", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True,
    )

In [13]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"A.": "choice here", "B.": "choice here", "C.": "choice here", "D.": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"A.": "choice here", "B.": "choice here", "C.": "choice here", "D.": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"A.": "choice here", "B.": "choice here", "C.": "choice here", "D.": "choice here"}, "correct": "correct answer"}}'

In [14]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "vocab_handout": vocab_handout,
            "number": 15,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

c:\Users\samue\anaconda3\envs\generative_ai\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

This is how you are to respond to this task: You are an expert seasoned examiner with the Apprenticeship and Industry Training (AIT) organization in Alberta, Canada. You have 20 years of experience 
preparing the preapprenticeship entrance exams for skilled trade participants. You are highly skilled and have high level expertise with minimal typographic mistakes or errors in the answers. You are 
skilled in generating multiple choice questions that contains both calculations, theory and conceptual questions. This tone must strongly reflect in the generated multiple choice questions. You are
provided with a vocabulary handout for the science subject. Using the    
 
Science: Lesson  5 – Gases and Properties of Gases  
 
Pressure : force applied perpendicular to the surface of an object per unit area.  
 
• Pressure of Solids:          𝑃𝑟𝑒𝑠𝑠𝑢𝑟𝑒 =𝐹𝑜𝑟𝑐𝑒
𝐴𝑟𝑒𝑎 
 
• Pressure of Liquids:     

In [50]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total Tokens: 17078
Prompt Tokens: 15693
Completion Tokens: 1385
Total Cost: 0.0263095


In [51]:
response

{'vocab_handout': "   \n \nScience: Lesson  5 – Gases and Properties of Gases  \n \nPressure : force applied perpendicular to the surface of an object per unit area.  \n \n• Pressure of Solids:          𝑃𝑟𝑒𝑠𝑠𝑢𝑟𝑒 =𝐹𝑜𝑟𝑐𝑒\n𝐴𝑟𝑒𝑎 \n \n• Pressure of Liquids:     𝑃𝑟𝑒𝑠𝑠𝑢𝑟𝑒 = 𝜌𝑔ℎ \no where 𝜌 is the density of the fluid  \no 𝑔 is acceleration due to gravity  \no ℎ is the height of the fluid above the object . \n \n• Atmospheric Pressure  is the pressure exerted by the atmosphere and is denoted \nby 𝑃𝑎𝑡𝑚. \n• Gauge Pressure : the pressure of a system above atmospheric pressure. The \nreadings from gauge pressure includes the weight of the atmosphere.  \n• Pneumatic Pressure : the pressure exerted by a pressurized gas. Pneumatic \nsystems work by compressing the gas and increasing pressure.  \n• Hydraulic Pressure : the pressure that is generated by a hydraulic fluid in a \nconfined space when it is subjected to an external force.  \n• Hydrostatic Pressure : the pressure exerted by a liquid at res

In [52]:
print(response["quiz"])

{
    "1": {
        "mcq": "What is the SI unit of pressure?",
        "options": {
            "A.": "Watts",
            "B.": "Joules",
            "C.": "Pascals",
            "D.": "Kelvin"
        },
        "correct": "C. Pascals"
    },
    "2": {
        "mcq": "Which type of pressure includes the weight of the atmosphere?",
        "options": {
            "A.": "Atmospheric Pressure",
            "B.": "Pneumatic Pressure",
            "C.": "Gauge Pressure",
            "D.": "Hydraulic Pressure"
        },
        "correct": "C. Gauge Pressure"
    },
    "3": {
        "mcq": "In the Ideal Gas Law equation PV = nRT, what does 'n' represent?",
        "options": {
            "A.": "Volume",
            "B.": "Pressure",
            "C.": "Number of moles of gas",
            "D.": "Ideal gas constant"
        },
        "correct": "C. Number of moles of gas"
    },
    "4": {
        "mcq": "What does Pascal's Principle state?",
        "options": {
            "A.": "Vo

In [53]:
print(response["review"])

As an expert exam reviewer with extensive experience in pre-apprenticeship entrance exams for AIT in Alberta, Canada, I have thoroughly evaluated the complexity, balance, accuracy, and correctness of the provided multiple-choice questions. The questions cover a diverse range of topics related to pressure, energy, and fluid mechanics. They include both theoretical and calculation-based questions, ensuring a balanced mix for the exam takers. The formulas and units used in the questions are accurate, and the provided answers have been verified for correctness. Overall, the quiz meets the required standard and is suitable for assessing the knowledge of prospective apprentices.


In [54]:
quiz = response.get("quiz")

In [55]:
quiz = json.loads(quiz)

In [56]:
len(quiz)

15

In [57]:
quiz_table_data = []

for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [58]:
quiz_table_data

[{'MCQ': 'What is the SI unit of pressure?',
  'Choices': 'A.: Watts | B.: Joules | C.: Pascals | D.: Kelvin',
  'Correct': 'C. Pascals'},
 {'MCQ': 'Which type of pressure includes the weight of the atmosphere?',
  'Choices': 'A.: Atmospheric Pressure | B.: Pneumatic Pressure | C.: Gauge Pressure | D.: Hydraulic Pressure',
  'Correct': 'C. Gauge Pressure'},
 {'MCQ': "In the Ideal Gas Law equation PV = nRT, what does 'n' represent?",
  'Choices': 'A.: Volume | B.: Pressure | C.: Number of moles of gas | D.: Ideal gas constant',
  'Correct': 'C. Number of moles of gas'},
 {'MCQ': "What does Pascal's Principle state?",
  'Choices': 'A.: Volume is inversely proportional to pressure | B.: Pressure is transmitted undiminished in an enclosed fluid | C.: Temperature is directly proportional to volume | D.: Density is inversely proportional to pressure',
  'Correct': 'B. Pressure is transmitted undiminished in an enclosed fluid'},
 {'MCQ': 'How does pressure change with altitude?',
  'Choices':

In [59]:
doc = Document()

# Add a heading
doc.add_heading('Sample Practice Quiz Questions', level=1)

# Write each question, options, and correct answer
for i, (key, value) in enumerate(quiz.items(), start=1):
    mcq = value["mcq"]
    options = "\n".join([f"{option}: {option_value}" for option, option_value in value["options"].items()])
    correct = value["correct"]
    # Add question number
    doc.add_paragraph(f'{i}. {mcq}', style='BodyText')
    # Add options
    doc.add_paragraph(options, style='BodyText')
    # Add correct answer
    doc.add_paragraph(f'Correct Answer: {correct}', style='BodyText')
    
    # Add spacing between questions
    if i < len(quiz):
        doc.add_paragraph('\n')

# Save the document
doc.save('Lesson_4-6_Quiz_Questions.docx')

c:\Users\samue\anaconda3\envs\generative_ai\lib\site-packages\docx\styles\styles.py:130: UserWarning: style lookup by style_id is deprecated. Use style name as key instead.
  return self._get_style_id_from_style(self[style_name], style_type)


In [89]:
quiz = pd.DataFrame(quiz_table_data)
quiz.head(10)

,MCQ,Choices,Correct
0,What is the basic property of matter that is t...,A.: Volume | B.: Mass | C.: Shape | D.: Weight,B. Mass
1,Which state of matter has a definite shape and...,A.: Solids | B.: Liquids | C.: Gases | D.: Plasma,A. Solids
2,What is the characteristic of gases in terms o...,"A.: Definite shape, definite volume | B.: No d...","C. No definite shape, no definite volume"
3,Which state of matter has molecules that are t...,A.: Solids | B.: Liquids | C.: Gases | D.: Plasma,A. Solids
4,What is the expansion behavior of gases when h...,A.: Contracts back to original size | B.: Expa...,D. Easily expands and contracts
5,At what temperature does a liquid begin to cha...,A.: Melting point | B.: Freezing point | C.: B...,C. Boiling point
6,Which type of property describes how a substan...,A.: Physical property | B.: Chemical property ...,B. Chemical property
7,What is the SI unit for force?,A.: Joules | B.: Watts | C.: Volts | D.: Newtons,D. Newtons
8,Which law of motion states that to every actio...,A.: Newton's First Law | B.: Newton's Second L...,C. Newton's Third Law
9,What is the SI unit for pressure?,A.: Pascals | B.: Watts | C.: Volts | D.: Ohms,A. Pascals


In [ ]:
quiz.info()

In [38]:
quiz.to_csv("machinelearning.csv", index=False)